In [1]:
# prepare the datasets
import pandas as pd

In [2]:
tob_data = pd.read_csv('tob_domain_only.csv')

tob_data.head(10)

,index,user,question,question_class,question_cluster,answer_v1,answer_v2,v1_x_v2
0,3,100,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822
1,4,100,what is structural testing,whQuestion,23,Structural testing (or White-Box testing) is a...,Structural testing (or White-Box testing) is a...,1.000000
2,5,100,what is a software technique,whQuestion,83,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,0.325859
3,6,100,what is system testing,whQuestion,29,System testing is a level of software testing ...,System testing is a level of software testing ...,1.000000
4,7,100,what is unit testing,whQuestion,136,Unit testing verifies the functioning in isola...,Unit testing verifies the functioning in isola...,1.000000
5,8,100,what is integration testing,whQuestion,263,Integration testing is the process of verifyin...,Integration testing is the process of verifyin...,1.000000
6,9,100,Can you give me an example of equivalence clas...,Clarify,213,Here is an example of an application of equiva...,"Sorry, I didn't understand your question, coul...",0.045760
7,10,100,How do I apply equivalence class partitioning,whQuestion,188,The equivalence partitioning criterion divides...,"Sorry, I didn't quite catch that",0.057074
8,11,100,What is verification,whQuestion,126,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,0.325859
9,12,100,what is validation,whQuestion,134,"Sorry, I still can't answer that.. Could you a...","Sorry, I didn't quite catch that",0.269803


In [3]:
gpt_data = pd.read_csv('gpt_answers_by_cluster_sample.csv')

gpt_data.head(10)

,Unnamed: 0,cluster,question,answer:gpt-3.5-turbo-16k
0,217,118,what is software testing,Software testing is the process of evaluating ...
1,1064,118,what is software testing,Software testing is the process of evaluating ...
2,601,118,what is software testing,Software testing is the process of evaluating ...
3,1675,118,What Is software testing,Software testing is the process of evaluating ...
4,1239,118,what is software testing,Software testing is the process of evaluating ...
5,1701,118,What is software test,Software testing is a process of evaluating a ...
6,1674,118,What Is software test,Software testing is the process of evaluating ...
7,270,118,What is software testing,Software testing is a process used to evaluate...
8,1398,162,What is a test case?,"A test case is a set of conditions or inputs, ..."
9,1562,162,i want to know what is a test case,A test case refers to a detailed and specific ...


In [4]:
def pick_random_gpt_answer_for_cluster(gpt_df, cluster):

    return gpt_df[gpt_df['cluster'] == cluster]['answer:gpt-3.5-turbo-16k'].sample(1).values[0]

In [5]:
pick_random_gpt_answer_for_cluster(gpt_data, 162)

'A test case is a set of specific conditions or steps devised to validate or verify the functionality of a software application or a system. It outlines the inputs, actions, and expected outcomes for a particular feature or functionality to ensure that the software functions correctly. Test cases are designed to cover a variety of scenarios, including both typical and exceptional situations, in order to thoroughly test the software and identify any potential defects or issues.'

In [6]:
def populate_with_random_gpt_from_cluster(gpt_df, cluster):

    return pick_random_gpt_answer_for_cluster(gpt_df, cluster)

In [7]:
tob_data['cluster_gpt_answer'] = tob_data.apply(lambda df: populate_with_random_gpt_from_cluster(gpt_data, df['question_cluster']), axis=1)

tob_data.head()

,index,user,question,question_class,question_cluster,answer_v1,answer_v2,v1_x_v2,cluster_gpt_answer
0,3,100,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822,Software testing is the process of evaluating ...
1,4,100,what is structural testing,whQuestion,23,Structural testing (or White-Box testing) is a...,Structural testing (or White-Box testing) is a...,1.000000,Structural testing is a type of software testi...
2,5,100,what is a software technique,whQuestion,83,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,0.325859,Software techniques are the methods and practi...
3,6,100,what is system testing,whQuestion,29,System testing is a level of software testing ...,System testing is a level of software testing ...,1.000000,System testing is a type of software testing t...
4,7,100,what is unit testing,whQuestion,136,Unit testing verifies the functioning in isola...,Unit testing verifies the functioning in isola...,1.000000,Unit testing is a software testing technique t...


In [8]:
# create SentenceTransformers encoder

from sentence_transformers import SentenceTransformer, util

encoder = SentenceTransformer('all-mpnet-base-v2')

In [9]:
# calculate and cache embedings

v1_embedings = encoder.encode(tob_data['answer_v1'])



In [10]:
v2_embedings = encoder.encode(tob_data['answer_v2'])


In [11]:
gpt_embedings = encoder.encode(tob_data['cluster_gpt_answer'])

In [12]:
def compare_embedings(base, alternate):
    cosine_similarity = util.cos_sim(base, alternate)

    return cosine_similarity
    



In [21]:
v1_v2 = [compare_embedings(v1, v2).item() for v1, v2 in zip(v1_embedings, v2_embedings)]

tob_data['v1_x_v2'] = v1_v2

In [22]:
v1_gpt = [compare_embedings(v1, gpt).item() for v1, gpt in zip(v1_embedings, gpt_embedings)]

tob_data['v1_x_gpt'] = v1_gpt

In [25]:
v2_gpt = [compare_embedings(v2, gpt).item() for v2, gpt in zip(v2_embedings, gpt_embedings)]

tob_data['v2_x_gpt'] = v2_gpt

In [26]:
tob_data

,index,user,question,question_class,question_cluster,answer_v1,answer_v2,v1_x_v2,cluster_gpt_answer,v1_x_gpt,v2_x_gpt
0,3,100,what is software testing,whQuestion,118,Software testing is an activity of software en...,Software testing is an activity of software en...,0.998822,Software testing is the process of evaluating ...,0.913018,0.914319
1,4,100,what is structural testing,whQuestion,23,Structural testing (or White-Box testing) is a...,Structural testing (or White-Box testing) is a...,1.000000,Structural testing is a type of software testi...,0.869358,0.869358
2,5,100,what is a software technique,whQuestion,83,"Sorry, I still can't answer that.. Could you a...",I don't understand what you mean,0.325859,Software techniques are the methods and practi...,0.130576,0.054581
3,6,100,what is system testing,whQuestion,29,System testing is a level of software testing ...,System testing is a level of software testing ...,1.000000,System testing is a type of software testing t...,0.975843,0.975843
4,7,100,what is unit testing,whQuestion,136,Unit testing verifies the functioning in isola...,Unit testing verifies the functioning in isola...,1.000000,Unit testing is a software testing technique t...,0.793993,0.793993
...,...,...,...,...,...,...,...,...,...,...,...
1817,2983,98,what is state transition testing,whQuestion,67,"It is a criteria within functional testing, bu...",I don't understand what you mean,0.136184,A state is a defined territory that has its ow...,0.039908,0.020009
1818,2984,98,what is the definition of state transition tes...,whQuestion,67,"It is a criteria within functional testing, bu...",I don't understand what you mean,0.136184,A state is a defined territory that has its ow...,0.039908,0.020009
1819,2985,98,what is graph testing,whQuestion,7,Software testing is an activity of software en...,I don't understand what you mean,-0.001124,Cause-effect graphing is a technique used in s...,0.420994,0.086755
1820,2986,98,what is graph matrix testing,whQuestion,7,"It is a criteria within functional testing, bu...",Software testing is an activity of software en...,0.409774,Cause-effect graphing is a technique used in s...,0.421582,0.420994


In [27]:
tob_data.to_csv('tob_gpt_similarity.csv')